In [8]:
import pandas as pd
pd.Series([1,2,2,3,4,5,4,3,2,5,6]).rank(pct=True)

0     0.090909
1     0.272727
2     0.272727
3     0.500000
4     0.681818
5     0.863636
6     0.681818
7     0.500000
8     0.272727
9     0.863636
10    1.000000
dtype: float64